In [1]:
from gensim.models import Word2Vec
from tensorflow import keras
import tensorflow as tf
from keras.models import load_model
import time
import glob
import os
import os.path
import random
import sys
import numpy as np
import nltk
nltk.download('punkt')
import matplotlib.pyplot as plt

from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop

sys.path.append('..')
sys.path.append('../..')



Using TensorFlow backend.
[nltk_data] Downloading package punkt to /Users/makoto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
predictionModel =  keras.models.load_model("../../models/seq2CommentPrediction.model")
maxLen=19

In [50]:
context2VecModel = Word2Vec.load("../../models/context2Vec.model")
comment2VecModel = Doc2Vec.load("../../models/comment2Vec.model")
context2SeqModel = keras.models.load_model("../../models/context2Seq.model")

In [8]:
#学習データのファイル群を取得
file = glob.glob("../../src/repositories2TokenWithComment/all.txt")
print(file)

['../../src/repositories2TokenWithComment/all.txt']


In [9]:
contexts = []
comments = []
f = open(file[0])
lines = f.readlines()
i=0
for line in lines:
    temp_data = line.replace('\n','')
    temp_context = temp_data.split(' ')[0:maxLen]
    temp_comment = " ".join(temp_data.split(' ')[(maxLen+1):])
    morph = nltk.word_tokenize(temp_comment)
    contexts.append(temp_context)
    comments.append(TaggedDocument(morph,[i]))
    i += 1
f.close()
print(len(contexts))

print(contexts[0])
print(contexts[0][0])

48920
['require', 'on_tstring_beg', 'on_tstring_content', 'on_tstring_end', 'require', 'on_tstring_beg', 'on_tstring_content', 'on_tstring_end', 'require', 'on_tstring_beg', 'on_tstring_content', 'on_tstring_end', 'require', 'on_tstring_beg', 'on_tstring_content', 'on_tstring_end', 'require', 'on_tstring_beg', 'on_tstring_content']
require


In [10]:
print(len(comments))
print(len(contexts))

print(contexts[0])


48920
48920
['require', 'on_tstring_beg', 'on_tstring_content', 'on_tstring_end', 'require', 'on_tstring_beg', 'on_tstring_content', 'on_tstring_end', 'require', 'on_tstring_beg', 'on_tstring_content', 'on_tstring_end', 'require', 'on_tstring_beg', 'on_tstring_content', 'on_tstring_end', 'require', 'on_tstring_beg', 'on_tstring_content']


In [11]:
print(len(contexts))
print(len(comments))

48920
48920


In [40]:
#95%を学習データに5%をテストデータに

trainNum = int((len(contexts)*9.99/10))
testNum = len(contexts)-trainNum
testData = contexts[trainNum:]
testDataAns = comments[trainNum:]

In [41]:
print(predictionModel.__dict__)

{'_self_setattr_tracking': True, '_trainable': True, '_stateful': False, '_build_input_shape': TensorShape([None, 30]), '_saved_model_inputs_spec': TensorSpec(shape=(None, 30), dtype=tf.float32, name='dense_input'), '_input_spec': None, '_supports_masking': True, '_name': 'sequential', '_activity_regularizer': None, '_trainable_weights': [], '_non_trainable_weights': [], '_updates': [], '_thread_local': <_thread._local object at 0x7f8671b57db0>, '_callable_losses': [], '_losses': [], '_metrics': [], '_metrics_lock': <unlocked _thread.lock object at 0x7f8690266fa8>, '_dtype_policy': <Policy "float32", loss_scale=None>, '_dtype_defaulted_to_floatx': False, '_compute_dtype_object': tf.float32, '_autocast': False, '_layers': [<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f869028b668>, <tensorflow.python.keras.layers.core.Dense object at 0x7f86610cc828>, <tensorflow.python.keras.layers.core.Activation object at 0x7f86610cc320>, <tensorflow.python.keras.layers.core.Dens

In [42]:
print(len(testData))
print(len(testDataAns))
print(np.array(testData).shape)
print(np.array(testDataAns).shape)

49
49
(49, 19)
(49, 2)


In [43]:

#全部の分散表現を得る

def distributedRepresentation(AllContexts,AllComments):
    contexts2Vec=[]
    comments2Vec=[]
    seq2Vec = []
    
    #ここでコンテキストの各単語の分散表現を得ている
    for i in range(len(AllContexts)):
        temp_ary = []
        for j in range(len(AllContexts[i])):
            temp_ary.append(np.array(context2VecModel.wv[AllContexts[i][j]]))
        temp_ary = np.array(temp_ary)
        contexts2Vec.append(temp_ary)
    print(len(contexts2Vec))
    
    #ここで各単語の分散表現を用いて文の分散表現を得ている
    for i in range(len(contexts2Vec)):
        print(i)
        seqVec = context2SeqModel.predict(np.array([contexts2Vec[i][:maxLen]]),verbose=2)[0]
        seq2Vec.append(np.array(seqVec))
        
    #ここでコメントの分散表現を得る
    comment2VecModel = Doc2Vec(documents=AllComments, vector_size=15,min_count=5,window=5,epochs=20,dm=0)
    for i in range(len(seq2Vec)):
        comments2Vec.append(comment2VecModel.docvecs[i])
    
    #ここでpredictionするモデルをする
    
    return seq2Vec, comments2Vec, comment2VecModel
    


In [44]:
seq2VecAns, comments2VecAns, comment2VecModelAns = distributedRepresentation(testData,testDataAns)

49
0
1/1 - 0s
1
1/1 - 0s
2
1/1 - 0s
3
1/1 - 0s
4
1/1 - 0s
5
1/1 - 0s
6
1/1 - 0s
7
1/1 - 0s
8
1/1 - 0s
9
1/1 - 0s
10
1/1 - 0s
11
1/1 - 0s
12
1/1 - 0s
13
1/1 - 0s
14
1/1 - 0s
15
1/1 - 0s
16
1/1 - 0s
17
1/1 - 0s
18
1/1 - 0s
19
1/1 - 0s
20
1/1 - 0s
21
1/1 - 0s
22
1/1 - 0s
23
1/1 - 0s
24
1/1 - 0s
25
1/1 - 0s
26
1/1 - 0s
27
1/1 - 0s
28
1/1 - 0s
29
1/1 - 0s
30
1/1 - 0s
31
1/1 - 0s
32
1/1 - 0s
33
1/1 - 0s
34
1/1 - 0s
35
1/1 - 0s
36
1/1 - 0s
37
1/1 - 0s
38
1/1 - 0s
39
1/1 - 0s
40
1/1 - 0s
41
1/1 - 0s
42
1/1 - 0s
43
1/1 - 0s
44
1/1 - 0s
45
1/1 - 0s
46
1/1 - 0s
47
1/1 - 0s
48
1/1 - 0s


In [45]:
context2VecModel.wv["if"]

array([-1.3342109 ,  0.17245089, -0.6987581 , -2.9237804 ,  1.8214276 ,
        0.3338277 , -0.04772862,  0.8789369 ,  2.3114316 , -2.5643578 ,
       -2.1370618 ,  1.0601604 , -3.4233077 , -2.6237285 ,  1.26362   ,
        0.5076294 ,  0.43313497,  4.1343055 ,  1.6192503 , -0.4854543 ,
       -1.4932156 , -0.19456454, -0.5381452 , -1.0571866 ,  0.08779773,
        2.4462073 ,  1.39535   ,  0.7677052 , -0.6336559 , -0.73793954],
      dtype=float32)

In [46]:
print(np.array(seq2VecAns).shape)
print(np.array(comments2VecAns).shape)

(49, 30)
(49, 15)


In [47]:
predictions = predictionModel.predict(np.array(seq2VecAns),batch_size=128)

In [48]:
print(len(predictions))
print(predictions[0])

49
[-0.15543951  0.1415143   0.11927209  0.05741173 -0.15560977  0.29037854
  0.19242388 -0.16947185 -0.21222205 -0.15362713  0.10942172 -0.01893082
  0.16041659 -0.19098864 -0.15574218  0.24273069  0.23745999  0.25079694
 -0.15873365  0.1084604   0.10612196  0.31464592  0.123501    0.27956814
  0.11708505  0.19031964  0.06992693  0.04750023 -0.19039904  0.19444995]


In [51]:
print(comment2VecModel.docvecs[0])
comment2VecModel.docvecs.most_similar(0)

[-0.14868934 -0.08465794  0.09190137 -0.01353424 -0.31690845  0.17415126
  0.10606049  0.17914826 -0.07863992  0.01682345  0.01169017  0.10625198
  0.20009005 -0.0842462   0.21384165 -0.20627783 -0.00624541  0.08933152
  0.2922954   0.1635617  -0.19888738  0.09782895  0.23663068  0.09092393
  0.1889024  -0.10144687 -0.07858437 -0.07050477 -0.09257706 -0.01705437]


[(176152, 0.9834058284759521),
 (6803, 0.9787476658821106),
 (51132, 0.9773884415626526),
 (310095, 0.975423276424408),
 (76626, 0.9752053022384644),
 (310767, 0.9746469855308533),
 (195464, 0.9736117720603943),
 (69946, 0.9726914167404175),
 (269751, 0.9724454879760742),
 (348486, 0.972417414188385)]

In [52]:
comment2VecModel.docvecs.__dict__

{'vectors': array([], shape=(0, 30), dtype=float64),
 'vocab': {},
 'vector_size': 30,
 'offset2doctag': [],
 'doctags': {},
 'max_rawint': 390416,
 'count': 390417,
 'mapfile_path': None,
 'vectors_docs_norm': array([[-1.82349369e-01, -1.03822641e-01,  1.12705834e-01, ...,
         -8.64655077e-02, -1.13534480e-01, -2.09151115e-02],
        [ 1.67161841e-02, -8.60683545e-02, -7.86724687e-02, ...,
         -3.14226836e-01, -4.08223569e-01,  3.04553926e-01],
        [-1.83010727e-01, -1.05653234e-01,  1.99078619e-01, ...,
          1.14437535e-01, -6.26554549e-01, -2.52258294e-04],
        ...,
        [-2.24501967e-01, -6.77460879e-02,  5.49420901e-02, ...,
          1.09152757e-01, -1.89459890e-01, -4.97081950e-02],
        [-2.13337317e-01, -2.40224048e-01, -4.41270359e-02, ...,
          9.61566269e-02, -4.82666403e-01, -1.77703634e-01],
        [-2.40856409e-01, -9.04069021e-02,  3.53296809e-02, ...,
          1.73511386e-01, -1.76564634e-01,  9.25698876e-03]], dtype=float32),
 '__

In [56]:
prediction_index = 40

In [57]:
def cosine_norm(model,ary,limit=10):
    temp_ary = []
    i = 0
    for v in model.docvecs.vectors_docs:
        distance = np.dot(v,ary)/(np.linalg.norm(v) * np.linalg.norm(ary))
        temp_ary.append([distance,i])
        i+=1
    temp_ary = sorted(temp_ary)
    temp_ary.reverse()
    return temp_ary[0:limit]

In [58]:
results = cosine_norm(comment2VecModel,predictions[prediction_index],10)

In [59]:
print(results)

[[0.66829973, 232278], [0.6638895, 321792], [0.6616788, 232251], [0.6597273, 33617], [0.6506997, 111838], [0.6465461, 142675], [0.641502, 254374], [0.6385877, 28235], [0.6364629, 81886], [0.62815785, 244311]]


In [67]:
results_index = []
for result in results:
    results_index.append(result[1])
print(results_index)
print(len(results_index))

[232278, 321792, 232251, 33617, 111838, 142675, 254374, 28235, 81886, 244311]
10


In [61]:
print(seq2VecAns[prediction_index])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]


In [62]:
print(testData[prediction_index])

['def', 'UNK', 'on_op', 'UNK', 'end', 'UNK', 'UNK', 'on_symbeg', 'webpacker', 'def', 'UNK', 'UNK', 'on_period', 'UNK', 'UNK', 'UNK', 'on_period', 'UNK', 'on_op']


In [63]:
print(testDataAns[prediction_index])

TaggedDocument(['#', 'Returns', 'true', 'if', 'the', 'compiled', 'packs', 'are', 'out', 'of', 'date', 'with', 'the', 'underlying', 'asset', 'files', '.'], [48911])


In [64]:
print(len(contexts))
print(len(comments))

48920
48920


In [69]:
for index in results_index:
    print()
    print(index)
    print('=======================')
    print(contexts[index])
    print(comments[index])
    print()


232278


IndexError: list index out of range